In [1]:
# Objet du lab1 : résumer des dialogues
# et se familiariser avec le prompt engineering (zero shot, one shot, few shot inference)
# Le prompt engineering permet de tester à moindre coût la performance d'un modèle sur une tâche donnée
# Utile pour pré-sélectionner un modèle avant de faire du fine-tuning sur 
# celui qui semble le plus performant pour la tâche envisagée

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade pip setuptools wheel

#!{sys.executable} -m pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1
!{sys.executable} -m pip install --disable-pip-version-check torch torchdata
!{sys.executable} -m pip install transformers==4.27.2 

# Transformers est un package créé par HuggingFace en accès libre

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/e0/63/b428aaca15fcd98c39b07ca7149e24bc14205ad0f1c80ba2b01835aedde1/pip-23.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 660.6 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.1 MB 1.6 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.1 MB 2.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.1 MB 2.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.1 MB 3.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.1 MB 3.4 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.1 MB 3.4 MB/s eta 0:00:01
   -------------------- ------------------- 1.1/2.1 MB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.2/2.1 MB 3.4 MB/

In [ ]:
!pip install -U datasets==2.14.6
!pip install fsspec==2023.9.2

In [3]:
import torch
print(torch.__version__)

2.0.1+cpu


In [ ]:
# 1 Set up Kernel and required dependencies

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [5]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('we love you'))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998704195022583}]


In [ ]:
# 2 Summarize dialogue without prompt engineering 

In [7]:
huggingface_dataset_name="knkarthick/dialogsum"
dataset=load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/V/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-931380d0e19583fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
#Print a couple of dialogues 

example_indices=[40,200]
dash_line='-'.join('' for x in range(100))

for i,index in enumerate(example_indices):
    print(dash_line)
    print('Exemple ',i+1)
    print(dash_line)
    print('INPUT DIALOGUE: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY: ')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Exemple  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
E

In [9]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Flan T5 peut faire beaucoup de choses, notamment du résumé

C:\Users\V\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\V\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [12]:
sentence="What time is it, Tom?"

sentence_encoded=tokenizer(sentence,return_tensors='pt')

sentence_decoded=tokenizer.decode(sentence_encoded["input_ids"][0],skip_special_tokens=True)

print('ENCODED SENTENCE: ')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE: ')
print(sentence_decoded)


ENCODED SENTENCE: 
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE: 
What time is it, Tom?


In [17]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    
    inputs=tokenizer(dialogue,return_tensors='pt')
    output=tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)
    
    print(dash_line)
    print('Exemple ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WHITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemple  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WHITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

------------------------------

In [ ]:
# 3 Summarize dialogue with an instruction prompt

In [ ]:
# 3.1 Zero-shot inference with an instruction prompt

In [20]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    
    prompt=f"""
Summarize the following conversation.

{dialogue}

Summary:
    """
    
    # Input constructed prompt instead of the dialogue
    inputs=tokenizer(prompt,return_tensors='pt')
    output=tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)
    
    print(dash_line)
    print('Exemple ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemple  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to

In [ ]:
# 3.2 Zero-shot inference with the prompt template from Flan-T5

In [ ]:
# On peut tester différentes manières de fournir l'instruction au modèle

In [21]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    
    prompt=f"""
Dialogue:
{dialogue}

What was going on?
    """
    
    # Input constructed prompt instead of the dialogue
    inputs=tokenizer(prompt,return_tensors='pt')
    output=tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)
    
    print(dash_line)
    print('Exemple ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT avec FLan-T5 pre-built prompt:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemple  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT avec FLan-T5 pre-built prompt:
Tom is lat

In [ ]:
# ça ne marche pas mieux...

In [ ]:
# 4 Summarize dialogue with one-shot and few-shot inference

In [ ]:
# 4.1 One-shot inference

In [ ]:
# On construit une liste d'exemples à fournir au modèle en début de prompt, avant de donner la discussion à résumer

In [25]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt=''
    for index in example_indices_full:
        dialogue=dataset['test'][index]['dialogue']
        summary=dataset['test'][index]['summary']
        prompt+=f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    # Ci-dessus, les trois sauts de lignes après {summary} sont importants pour aider Flan-T5 à comprendre. 
    # D'autres modèles peuvent avoir d'autres motifs préférentiels
    
    dialogue=dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt+=f"""
Dialogue:

{dialogue}

What was going on?
"""
    return prompt

In [27]:
example_indices_full=[40]
example_index_to_summarize=200

one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [28]:
summary=dataset['test'][example_index_to_summarize]['summary']

inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)


print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [ ]:
# C'est un peu mieux...

In [ ]:
# 4.2 Few-shot inference

In [29]:
example_indices_full=[40,80,120]
example_index_to_summarize=200

few_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

In [30]:
summary=dataset['test'][example_index_to_summarize]['summary']

inputs=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


In [ ]:
# 5 Generative configuration parameters for inference

In [32]:
generation_config=GenerationConfig(max_new_tokens=50)
#generation_config=GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.8)

inputs=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],generation_config)[0],skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


In [33]:
generation_config=GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.8)

inputs=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],generation_config)[0],skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
People are thinking up something to do on the computer in next few days. They are looking to make changes and improve their system systems and computers which are both already developed. Some people consider adding a painting program to the program. Another idea comes Regular


In [ ]:
# Bac à sable

In [43]:
generation_config=GenerationConfig(max_new_tokens=50,do_sample=True,temperature=0.7)

inputs=tokenizer("What is the average temperature in Paris in October ?",return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],generation_config)[0],skip_special_tokens=True)

print(dash_line)
print('Question:\n"What is the average temperature in Paris in October, in °C ?"')
print(dash_line)
print(f'Réponse:\n{output}')

---------------------------------------------------------------------------------------------------
Question:
"What is the average temperature in Paris in October, in °C ?"
---------------------------------------------------------------------------------------------------
Réponse:
-45 ° celsius
